In [36]:
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.sql.functions.udf
import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.sql.functions.udf
import org.apache.spark.ml.feature.VectorAssembler


In [37]:
val weather_org_df = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("weatherAUS.csv")

weather_org_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [38]:
weather_org_df.registerTempTable("Aus_Rain_Data")

In [39]:
val df2 = spark.sql("select Date, Location, MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustDir, WindGustSpeed, WindDir9am, WindDir3pm, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, cast(Pressure9am as Double), cast(Pressure3pm as Double), Cloud9am, Cloud3pm, Temp9am, Temp3pm, RainToday, RISK_MM, RainTomorrow from Aus_Rain_Data")

df2: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [40]:
val min_P9am = df2.agg(min("Pressure9am") as "min_P9am").select(col("min_P9am")).first.getDouble(0)
val max_P9am = df2.agg(max("Pressure9am") as "max_P9am").select(col("max_P9am")).first.getDouble(0)

val min_P3pm = df2.agg(min("Pressure3pm") as "min_P3pm").select(col("min_P3pm")).first.getDouble(0)
val max_P3pm = df2.agg(max("Pressure3pm") as "max_P3pm").select(col("max_P3pm")).first.getDouble(0)

min_P9am: Double = 980.5
max_P9am: Double = 1041.0
min_P3pm: Double = 977.1
max_P3pm: Double = 1039.6


In [41]:
val minPRange = 970.0    //from the report
val maxPRange = 1050.0   //from the report

minPRange: Double = 970.0
maxPRange: Double = 1050.0


\begin{equation}
  Rescaled_{9am} = \frac{Pressure_i - Pressure_{min9am}}{Pressure_{max9am} - Pressure_{min9am}} * (maxPressureRange - minPressureRange) + minPressureRange
\end{equation}

In [42]:
val df3 = df2.withColumn("Pressure3pm_Normalised", 
    (((df2("Pressure3pm") - min_P3pm) / (max_P3pm -min_P3pm)) * (maxPRange - minPRange)) + minPRange)

df3: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 23 more fields]


In [43]:
val df4 = df3.withColumn("Pressure9am_Normalised", 
    (((df2("Pressure9am") - min_P9am) / (max_P9am -min_P9am)) * (maxPRange - minPRange)) + minPRange)

df4: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 24 more fields]


In [44]:
!rm -r /home/normalised/

rm: cannot remove '/home/normalised/': No such file or directory



In [45]:
df4.coalesce(1)
      .write
      .option("header","true")
      .option("sep",",")
      .mode("overwrite")
      .csv("normalised")

In [46]:
!ls /home/normalised

_SUCCESS  part-00000-c19a64f7-fadd-49e0-9c47-7b81e07f5735-c000.csv



In [47]:
!mv /home/normalised/*.csv /home/

In [48]:
!mv /home/part*.csv /home/weatherAUS_N.csv

In [49]:
!wc -l   weatherAUS_N.csv
!wc -l   weatherAUS.csv

142194 weatherAUS_N.csv

142194 weatherAUS.csv



In [50]:
!rm -r /home/normalised/